# 02 - Proyecto individual: Feature Engineering

## Instalamos las librrerías requeridas.

Para realizar el análisis de sentimientos del los reviews del dataset `reviews`.

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00


## Montamos la partición de Google Drive.

Para acceder al dataset preprocesado `reviews`.

In [2]:
%cd /content/drive/MyDrive/AI Projects/PI01

/content/drive/MyDrive/AI Projects/PI01


## Cargamos las liberías necesarias.

In [3]:
import pandas as pd
import numpy as np
import transformers

## Cargamos el dataset `reviews`.

In [17]:
reviews_df = pd.read_parquet('datasets/reviews.parquet')

In [18]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   item_id    59305 non-null  object
 2   recommend  59305 non-null  object
 3   review     59305 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [19]:
reviews_df.head()

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...


### Verificamos que no hallan valores nulos.

In [20]:
reviews_df.review.isnull().sum()

28

In [21]:
reviews_df = reviews_df.query('review.notnull()')

In [22]:
reviews_df.review.isnull().sum()

0

## Instanciamos el clasificador de setimientos.

In [10]:
sentiment_pipeline = transformers.pipeline(model='mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis',
                                           max_length=512, truncation=True, device=0)

### Testeamos el modelo clasificador antes

In [23]:
np.random.seed(42)
reviews_sample = reviews_df.review.sample(5).tolist()
reviews_sample

['PEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PEW PEW PEWPEW PEW PE

In [24]:
prediction = sentiment_pipeline(reviews_sample)
prediction

[{'label': 'neutral', 'score': 0.9993152618408203},
 {'label': 'positive', 'score': 0.9988011121749878},
 {'label': 'neutral', 'score': 0.9997655749320984},
 {'label': 'neutral', 'score': 0.9997170567512512},
 {'label': 'neutral', 'score': 0.9989659786224365}]

### Aplicamos el analizador de sentimientos a todo el campo `review`.

In [25]:
reviews_df['sentiment'] = sentiment_pipeline(reviews_df.review.tolist())

In [27]:
def helper_fn(item):
  polarity = 0
  label = item['label']
  if label == 'positive':
    polarity = 2
  elif label == 'negative':
    polarity = 0
  elif label == 'neutral':
    polarity = 1

  return polarity

reviews_df['sentiment'] = reviews_df.sentiment.map(helper_fn)
reviews_df['sentiment'].value_counts()

1    46098
2    10072
0     3135
Name: sentiment, dtype: int64

### Guardamos el dataset actualizado.

In [28]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59305 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  object
 2   recommend  59305 non-null  object
 3   review     59305 non-null  object
 4   sentiment  59305 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


In [29]:
reviews_columns = ['item_id', 'user_id', 'recommend', 'sentiment']

In [30]:
reviews_df[reviews_columns].to_parquet('datasets/reviews_stt.parquet', index=False)